In [1]:
import numpy as np
from scipy.stats import poisson

Note: If you run infer it will work just find as all global variables are defined through it. If not  you need to define the global variables:

lambda0 (lambda for zvalue 0) \
lambda1 (lambda for zvalue 1) \
x_mat (vector of lambda0 and lambda1) \
z_mat (z-probabilites given c) \
lambda_mat (c_j probabilities given c_i)

In [ ]:
def calc_z_prob(x_value, lambdas):
    lambda0_prob = poisson.pmf(x_value, lambdas[0])
    lambda1_prob = poisson.pmf(x_value, lambdas[1])

    return np.array([lambda0_prob,lambda1_prob]).reshape(2,-1).T

def normalize_2darr(data):
    return  (1/np.sum(data, axis=1).reshape(-1,1) * data)
    
def normalize_1darr(data):
    return 1/np.sum(data) * data

def forward_message_pass(X, gamma_mat, z_mat, lambdas):

    #reading the values of T and n off of X
    T = X.shape[0]
    n = X.shape[1]

    #defining the final result matrices
    c_prob_mat = np.zeros((T, 3)).astype(float)
    z_prob_mat = np.zeros((T, n, 2)).astype(float)
    z_to_c_tot_mat = np.zeros((T,3)).astype(float)

    #looping over the time steps
    for t in range(T):

        #calculating all the x-z factors from the observed X
        z_givenx_probs = calc_z_prob(X[t], lambdas)
        z_prob_mat[t] = normalize_2darr(z_givenx_probs)

        #calculating the z_i-c factors
        c_givenz_probs = z_prob_mat[t] @ z_mat.T

        # print(c_givenz_probs.shape)
        c_givenz_probs_normalized = normalize_2darr(c_givenz_probs)
    

        #aggregating the z_i factors
        c_givenz_tot = np.product(c_givenz_probs_normalized, axis=0)
        c_givenz_tot_normalized = normalize_1darr(c_givenz_tot)
        z_to_c_tot_mat[t] = c_givenz_tot_normalized

        #updating the c_i-c_j factors
        if(t==0):
            c_prob_mat[t] = normalize_1darr(c_givenz_tot_normalized @ gamma_mat)
        
        else:
            outer = np.outer(c_prob_mat[t-1], c_givenz_tot_normalized) * gamma_mat
            summed = np.sum(outer, axis = 0)
            c_prob_mat[t] = normalize_1darr(summed)
    
    return c_prob_mat, z_prob_mat , z_to_c_tot_mat

def backward_message_pass(c_T, gamma_mat, z_mat, T, n): 
    
    #defining the final result matrices
    c_backprob_mat = np.zeros((T, 3)).astype(float)
    z_backprob_mat = np.zeros((T, n, 2)).astype(float)

    #setting c_T value
    c_backprob_mat[-1] = c_T

    #looping over timesteps in reverse order
    for t in range(T-1,-1,-1):

        #updating the factors
        if t-1 >= 0:
            c_backprob_mat[t-1] = normalize_1darr(c_backprob_mat[t] @ gamma_mat.T)
        z_backprob_mat[t] = normalize_1darr(c_backprob_mat[t] @  z_mat)

    return c_backprob_mat, z_backprob_mat

def infer(X, alpha, beta, gamma, lambdas, soft = False):
    
    #reading the values of T and n off of X
    T = X.shape[0]
    n = X.shape[1]

    #defining the probability matrices as global variables
    gamma_mat = np.array([[1-gamma,0,gamma],[0,1-gamma,gamma],[beta/2,beta/2,1-beta]])
    z_mat = np.array([[alpha, 1-alpha],[1-alpha, alpha],[0.5,0.5]])

    #finding the clique beliefs
    c_forward_probs, z_forward_probs, z_to_c_tot_mat = forward_message_pass(X, gamma_mat, z_mat, lambdas)
    c_backward_probs, z_backward_probs = backward_message_pass(c_forward_probs[-1], gamma_mat, z_mat, T=T, n=n)

    #finding the inferred values via MAP

    c_beliefs = []
    for t in range(len(c_forward_probs)):

        if(t == 0):
            c_beliefs.append((z_to_c_tot_mat[t] * c_backward_probs[t]) * gamma_mat)
        
        elif (t == len(c_forward_probs)-1):
            c_beliefs.append(z_to_c_tot_mat[t] * gamma_mat)
        
        else:
            c_beliefs.append(np.outer(c_forward_probs[t] , (z_to_c_tot_mat[t] * c_backward_probs[t])) * gamma_mat)
    
    c_beliefs = np.array(c_beliefs)

     
    # c_belief = []
    # for c_for,c_back in zip(c_forward_probs,c_backward_probs):
    #     outer = np.outer(c_for, c_back)
    #     c_belief.append(np.sum(outer, axis = 1))

    z_belief = []
    for z_for,z_back in zip(z_forward_probs,z_backward_probs):
        n = []
        for i in range(len(z_for)):
            outer = np.outer(z_for[i], z_back[i])
            n.append(np.sum(outer, axis = 1))

        z_belief.append(n)

    c_inferred = c_beliefs.sum(axis=2).argmax(axis=1)
    z_inferred = (np.array(z_belief)).argmax(axis=2)

    return c_inferred, z_inferred